# Lag Feature (전주 판매점수) 효과 테스트

## 분석 개요

### 목적
**과거 판매 실적(lag feature)**이 현재 판매 예측에 미치는 영향을 분석하여, 기존 피처(바이럴 지수, 카테고리 등) 대비 예측력을 비교

### 핵심 질문
1. 전주 판매점수(y_lag1)가 다음 주 판매를 얼마나 잘 예측하는가?
2. 기존 43개 피처 vs lag 피처, 어느 것이 더 강력한가?
3. lag 피처만으로 충분한 예측이 가능한가?

### 분석 방법
- **회귀 분석**: LightGBM으로 판매점수 예측 (R², MAE)
- **분류 분석**: 베스트셀러 진입 여부 예측 (F1, AUC)
- **피처셋 비교**: 기존 피처만 / +lag1 / +lag1~4 / lag만

---

## 데이터 설명

### 1. ML 데이터셋 (books_ml_dataset_v3.csv)
| 항목 | 설명 |
|------|------|
| 레코드 수 | 7,191개 (책 × 주차) |
| 기간 | 2024년 1월 ~ 2025년 12월 |
| 단위 | 주간 (ymw: 년월주) |

### 2. 타겟 변수 (y_sales_score)
| 값 | 의미 |
|-----|------|
| 0 | 해당 주 베스트셀러 미진입 |
| 1~20 | 베스트셀러 순위 점수 (21-순위) |

### 3. 피처 구성 (47개)

#### 기존 피처 (43개)
| 그룹 | 피처 | 설명 |
|------|------|------|
| 바이럴 지수 | viral_index, viral_index_smoothed | 뉴스 기반 관심도 |
| 카테고리 | category_1 ~ category_10 | 도서 분류 (원핫) |
| 카테고리×바이럴 | category_X_x_viral_index | 교차항 |
| Prophet 예측 | prophet_forecast | 시계열 예측값 |

#### Lag 피처 (4개)
| 피처 | 설명 |
|------|------|
| y_lag1 | 전주 판매점수 |
| y_lag2 | 2주 전 판매점수 |
| y_lag3 | 3주 전 판매점수 |
| y_lag4 | 4주 전 판매점수 |

---

## 데이터 한계점

### 1. Lag 피처의 구조적 한계
| 한계 | 영향 |
|------|------|
| **신규 도서 예측 불가** | 과거 데이터 없는 책은 lag 피처 사용 불가 |
| **Cold Start 문제** | 첫 4주는 lag1~4 결측 발생 |
| **자기상관 의존** | 모델이 "관성"에만 의존할 위험 |

### 2. 데이터 불균형
- 베스트셀러 진입(y>0): 전체의 약 23%
- 비진입(y=0): 약 77%
- → class_weight='balanced' 적용으로 보완

### 3. 순환 예측 한계
- 실제 서비스에서 y_lag1은 "전주 실제값"
- 2주 이상 예측 시 예측값을 lag로 사용해야 함 → 오차 누적

---

## 보완점 및 향후 계획

### 1. 신규 도서 대응
- **출시 첫 주 전용 모델**: lag 없이 예측하는 별도 모델
- **유사 도서 기반**: 같은 저자/카테고리의 과거 실적 참조
- **사전 관심도**: 출시 전 뉴스/SNS 버즈 활용

### 2. 예측 안정성 강화
- **앙상블**: lag 모델 + 외부변수 모델 결합
- **Confidence Interval**: 예측 불확실성 제공
- **이상치 탐지**: 급등/급락 케이스 별도 처리

### 3. 실시간 적용
- **주간 자동 업데이트**: 새 데이터 반영
- **A/B 테스트**: 예측 기반 의사결정 효과 검증

---

## 예상 결과 요약

> **가설**: 전주 판매점수(y_lag1)가 가장 강력한 예측 변수일 것

| 예상 | 근거 |
|------|------|
| y_lag1 상관계수 > 0.8 | 베스트셀러의 "관성" 효과 |
| lag 피처만으로 R² > 0.6 | 과거 성과가 미래 성과 결정 |
| 기존 피처 단독 R² < 0.4 | 바이럴 지수의 약한 예측력 |

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (f1_score, roc_auc_score, r2_score, mean_absolute_error,
                             precision_score, recall_score, accuracy_score)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.svm import SVR, SVC
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from scipy import stats
import lightgbm as lgb
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

## 1. 데이터 로드 및 Lag 피처 생성

In [2]:
df = pd.read_csv('books_ml_dataset_v3.csv')
df['ymw'] = df['ymw'].astype(str)
df = df.sort_values(['product_code', 'ymw']).reset_index(drop=True)

# Lag 피처 생성 (책별로)
for lag in [1, 2, 3, 4]:
    df[f'y_lag{lag}'] = df.groupby('product_code')['y_sales_score'].shift(lag)

print(f'원본 데이터: {len(df):,}개')
print(f'\n[Lag 피처 결측치]')
for lag in [1, 2, 3, 4]:
    na = df[f'y_lag{lag}'].isna().sum()
    print(f'y_lag{lag}: {na}개 ({na/len(df)*100:.1f}%)')

# 결측치 제거
df_lag = df.dropna(subset=['y_lag1', 'y_lag2', 'y_lag3', 'y_lag4']).reset_index(drop=True)
print(f'\n결측치 제거 후: {len(df_lag):,}개')

원본 데이터: 7,191개

[Lag 피처 결측치]
y_lag1: 141개 (2.0%)
y_lag2: 282개 (3.9%)
y_lag3: 423개 (5.9%)
y_lag4: 564개 (7.8%)

결측치 제거 후: 6,627개


## 2. 상관관계 분석

In [3]:
y = df_lag['y_sales_score']
feature_cols = [c for c in df_lag.columns if c not in ['product_code', 'ymw', 'y_sales_score'] and not c.startswith('y_lag')]

print('[Lag 피처 vs 타겟 상관관계]')
print(f'{"피처":<12} {"r":<10} {"해석"}')
print('-'*35)
for lag in [1, 2, 3, 4]:
    corr, _ = stats.pearsonr(df_lag[f'y_lag{lag}'], y)
    print(f'y_lag{lag:<7} {corr:<10.4f} {"강함" if corr > 0.5 else "중간"}')

# 기존 피처 최고 상관관계
best_corr, best_feat = 0, ''
for col in feature_cols:
    corr, _ = stats.pearsonr(df_lag[col], y)
    if abs(corr) > abs(best_corr):
        best_corr, best_feat = corr, col

print(f'\n[기존 피처 최고] {best_feat}: r = {best_corr:.4f}')
print(f'\n→ y_lag1 (r=0.885)이 기존 최고 피처 (r={best_corr:.3f})보다 {0.885/abs(best_corr):.0f}배 강함')

[Lag 피처 vs 타겟 상관관계]
피처           r          해석
-----------------------------------
y_lag1       0.8850     강함
y_lag2       0.7759     강함
y_lag3       0.6677     강함
y_lag4       0.5648     강함

[기존 피처 최고] category_10: r = 0.0935

→ y_lag1 (r=0.885)이 기존 최고 피처 (r=0.093)보다 9배 강함


## 3. 피처셋별 성능 비교

In [4]:
X = df_lag[feature_cols + ['y_lag1', 'y_lag2', 'y_lag3', 'y_lag4']]
y = df_lag['y_sales_score']
y_class = (y > 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_class, y_test_class = (y_train > 0).astype(int), (y_test > 0).astype(int)

feature_sets = {
    '기존 피처만 (43개)': feature_cols,
    '기존 + y_lag1': feature_cols + ['y_lag1'],
    '기존 + y_lag1~4': feature_cols + ['y_lag1', 'y_lag2', 'y_lag3', 'y_lag4'],
    'y_lag만 (1~4)': ['y_lag1', 'y_lag2', 'y_lag3', 'y_lag4'],
    'y_lag1만': ['y_lag1'],
}

results = []
print(f'{"피처셋":<22} {"R2":<10} {"MAE":<10} {"F1":<10} {"AUC":<10}')
print('-'*62)

for name, features in feature_sets.items():
    # 회귀
    reg = lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    reg.fit(X_train[features], y_train)
    y_pred = np.maximum(reg.predict(X_test[features]), 0)
    
    # 분류
    clf = lgb.LGBMClassifier(n_estimators=100, class_weight='balanced', random_state=42, verbose=-1)
    clf.fit(X_train[features], y_train_class)
    y_prob = clf.predict_proba(X_test[features])[:, 1]
    y_pred_c = clf.predict(X_test[features])
    
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    f1 = f1_score(y_test_class, y_pred_c)
    auc = roc_auc_score(y_test_class, y_prob)
    
    results.append({'name': name, 'r2': r2, 'mae': mae, 'f1': f1, 'auc': auc})
    print(f'{name:<22} {r2:<10.4f} {mae:<10.4f} {f1:<10.4f} {auc:<10.4f}')

피처셋                    R2         MAE        F1         AUC       
--------------------------------------------------------------
기존 피처만 (43개)           0.3450     1.7990     0.6047     0.8277    
기존 + y_lag1            0.6937     0.7484     0.9446     0.9465    
기존 + y_lag1~4          0.6971     0.7495     0.9432     0.9473    
y_lag만 (1~4)           0.6905     0.7722     0.9490     0.9544    
y_lag1만                0.7009     0.7383     0.9490     0.9513    


## 4. 개선율 분석

In [ ]:
base = results[0]
with_lag1 = results[1]
only_lag = results[3]

print('[기존 피처 → 기존 + y_lag1 개선율]')
print(f'R²:  {base["r2"]:.4f} → {with_lag1["r2"]:.4f} ({(with_lag1["r2"]-base["r2"])/abs(base["r2"])*100:+.1f}%)')
print(f'MAE: {base["mae"]:.4f} → {with_lag1["mae"]:.4f} ({(with_lag1["mae"]-base["mae"])/base["mae"]*100:+.1f}%)')
print(f'F1:  {base["f1"]:.4f} → {with_lag1["f1"]:.4f} ({(with_lag1["f1"]-base["f1"])/base["f1"]*100:+.1f}%)')
print(f'AUC: {base["auc"]:.4f} → {with_lag1["auc"]:.4f} ({(with_lag1["auc"]-base["auc"])/base["auc"]*100:+.1f}%)')

print(f'\n[y_lag만 사용 vs 기존 피처만]')
print(f'R²:  {only_lag["r2"]:.4f} vs {base["r2"]:.4f} → y_lag만으로 {only_lag["r2"]/base["r2"]:.1f}배 성능')

## 5. 다중 모델 성능 비교 (회귀)

In [ ]:
# 회귀 모델 정의
reg_models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5, n_jobs=-1),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
}

# 스케일링이 필요한 모델
needs_scaling_reg = ['Linear Regression', 'Ridge', 'Lasso', 'KNN']

# 데이터 준비
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# 피처셋 (간소화)
reg_feature_sets = {
    '기존 피처만': feature_cols,
    '기존 + y_lag1': feature_cols + ['y_lag1'],
    'y_lag1만': ['y_lag1'],
}

# 모든 조합 테스트
reg_results = []

for model_name, model in reg_models.items():
    for feat_name, feat_cols_list in reg_feature_sets.items():
        # 스케일링 여부
        if model_name in needs_scaling_reg:
            X_tr, X_te = X_train_scaled[feat_cols_list], X_test_scaled[feat_cols_list]
        else:
            X_tr, X_te = X_train[feat_cols_list], X_test[feat_cols_list]
        
        # 학습 및 예측
        reg = model.__class__(**model.get_params())
        reg.fit(X_tr, y_train)
        y_pred = np.maximum(reg.predict(X_te), 0)
        
        reg_results.append({
            'model': model_name,
            'features': feat_name,
            'r2': r2_score(y_test, y_pred),
            'mae': mean_absolute_error(y_test, y_pred),
        })

reg_results_df = pd.DataFrame(reg_results)
print('[회귀 모델 성능 비교]')
print('='*70)

In [ ]:
# 피벗 테이블로 비교
pivot_r2 = reg_results_df.pivot(index='model', columns='features', values='r2')
pivot_r2 = pivot_r2[['기존 피처만', '기존 + y_lag1', 'y_lag1만']]
pivot_r2['개선율(%)'] = ((pivot_r2['기존 + y_lag1'] - pivot_r2['기존 피처만']) / pivot_r2['기존 피처만'].abs() * 100).round(1)
pivot_r2 = pivot_r2.sort_values('기존 + y_lag1', ascending=False)

print('[모델별 R² 점수 비교]')
print(pivot_r2.round(4).to_string())

# 최고 성능 모델
best_reg = reg_results_df.loc[reg_results_df['r2'].idxmax()]
print(f'\n★ 최고 성능: {best_reg["model"]} + {best_reg["features"]} (R²={best_reg["r2"]:.4f})')

## 6. 다중 모델 성능 비교 (분류)

In [ ]:
# 분류 모델 정의
clf_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
    'SVM (RBF)': SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42),
    'LightGBM': lgb.LGBMClassifier(n_estimators=100, class_weight='balanced', random_state=42, verbose=-1),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, scale_pos_weight=3, random_state=42, verbosity=0),
}

needs_scaling_clf = ['Logistic Regression', 'KNN', 'SVM (RBF)']

# 모든 조합 테스트
clf_results = []

for model_name, model in clf_models.items():
    for feat_name, feat_cols_list in reg_feature_sets.items():
        if model_name in needs_scaling_clf:
            X_tr, X_te = X_train_scaled[feat_cols_list], X_test_scaled[feat_cols_list]
        else:
            X_tr, X_te = X_train[feat_cols_list], X_test[feat_cols_list]
        
        clf = model.__class__(**model.get_params())
        clf.fit(X_tr, y_train_class)
        y_pred_c = clf.predict(X_te)
        
        try:
            y_prob = clf.predict_proba(X_te)[:, 1]
            auc = roc_auc_score(y_test_class, y_prob)
        except:
            auc = np.nan
        
        clf_results.append({
            'model': model_name,
            'features': feat_name,
            'f1': f1_score(y_test_class, y_pred_c),
            'auc': auc,
            'precision': precision_score(y_test_class, y_pred_c),
            'recall': recall_score(y_test_class, y_pred_c),
        })

clf_results_df = pd.DataFrame(clf_results)
print('[분류 모델 성능 비교]')
print('='*70)

In [ ]:
# 피벗 테이블로 비교
pivot_f1 = clf_results_df.pivot(index='model', columns='features', values='f1')
pivot_f1 = pivot_f1[['기존 피처만', '기존 + y_lag1', 'y_lag1만']]
pivot_f1['개선율(%)'] = ((pivot_f1['기존 + y_lag1'] - pivot_f1['기존 피처만']) / pivot_f1['기존 피처만'] * 100).round(1)
pivot_f1 = pivot_f1.sort_values('기존 + y_lag1', ascending=False)

print('[모델별 F1 점수 비교]')
print(pivot_f1.round(4).to_string())

# 최고 성능 모델
best_clf = clf_results_df.loc[clf_results_df['f1'].idxmax()]
print(f'\n★ 최고 성능: {best_clf["model"]} + {best_clf["features"]} (F1={best_clf["f1"]:.4f}, AUC={best_clf["auc"]:.4f})')

## 7. 모델 성능 시각화

In [ ]:
# 히트맵 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 회귀 R² 히트맵
ax1 = axes[0]
pivot_r2_plot = reg_results_df.pivot(index='model', columns='features', values='r2')
pivot_r2_plot = pivot_r2_plot[['기존 피처만', '기존 + y_lag1', 'y_lag1만']]
pivot_r2_plot = pivot_r2_plot.sort_values('기존 + y_lag1', ascending=False)
sns.heatmap(pivot_r2_plot, annot=True, fmt='.3f', cmap='RdYlGn', ax=ax1, vmin=0, vmax=0.8)
ax1.set_title('회귀 모델 R² 비교')
ax1.set_xlabel('')
ax1.set_ylabel('')

# 분류 F1 히트맵
ax2 = axes[1]
pivot_f1_plot = clf_results_df.pivot(index='model', columns='features', values='f1')
pivot_f1_plot = pivot_f1_plot[['기존 피처만', '기존 + y_lag1', 'y_lag1만']]
pivot_f1_plot = pivot_f1_plot.sort_values('기존 + y_lag1', ascending=False)
sns.heatmap(pivot_f1_plot, annot=True, fmt='.3f', cmap='RdYlGn', ax=ax2, vmin=0.3, vmax=1.0)
ax2.set_title('분류 모델 F1 비교')
ax2.set_xlabel('')
ax2.set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# 피처셋별 평균 성능 요약
print('[피처셋별 평균 성능 요약]')
print('='*60)

# 회귀
reg_summary = reg_results_df.groupby('features')['r2'].mean()
print('\n[회귀 - 평균 R² (8개 모델)]')
for feat in ['기존 피처만', '기존 + y_lag1', 'y_lag1만']:
    print(f'  {feat}: {reg_summary[feat]:.4f}')

# 분류
clf_summary = clf_results_df.groupby('features')[['f1', 'auc']].mean()
print('\n[분류 - 평균 F1/AUC (7개 모델)]')
for feat in ['기존 피처만', '기존 + y_lag1', 'y_lag1만']:
    print(f'  {feat}: F1={clf_summary.loc[feat, "f1"]:.4f}, AUC={clf_summary.loc[feat, "auc"]:.4f}')

# 개선율
r2_improvement = (reg_summary['기존 + y_lag1'] - reg_summary['기존 피처만']) / reg_summary['기존 피처만'] * 100
f1_improvement = (clf_summary.loc['기존 + y_lag1', 'f1'] - clf_summary.loc['기존 피처만', 'f1']) / clf_summary.loc['기존 피처만', 'f1'] * 100

print(f'\n[y_lag1 추가 시 평균 개선율]')
print(f'  R²: +{r2_improvement:.1f}%')
print(f'  F1: +{f1_improvement:.1f}%')

## 8. LightGBM 피처셋별 시각화

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# R² 비교
ax1 = axes[0]
names = [r['name'] for r in results]
r2_vals = [r['r2'] for r in results]
colors = ['#3498db' if 'lag' not in n else '#e74c3c' for n in names]
ax1.barh(names, r2_vals, color=colors)
ax1.set_xlabel('R²')
ax1.set_title('회귀 성능 (R²)')
ax1.axvline(x=base['r2'], color='gray', linestyle='--', alpha=0.5, label='기존 피처')

# F1 비교
ax2 = axes[1]
f1_vals = [r['f1'] for r in results]
ax2.barh(names, f1_vals, color=colors)
ax2.set_xlabel('F1 Score')
ax2.set_title('분류 성능 (F1)')
ax2.axvline(x=base['f1'], color='gray', linestyle='--', alpha=0.5, label='기존 피처')

plt.tight_layout()
plt.show()

In [ ]:
# y_lag1 vs y 산점도
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(df_lag['y_lag1'], df_lag['y_sales_score'], alpha=0.3, s=10)
ax.plot([0, df_lag['y_sales_score'].max()], [0, df_lag['y_sales_score'].max()], 'r--', label='y=x')
ax.set_xlabel('전주 판매점수 (y_lag1)')
ax.set_ylabel('현재 판매점수 (y)')
ax.set_title(f'전주 vs 현재 판매점수 (r = {stats.pearsonr(df_lag["y_lag1"], df_lag["y_sales_score"])[0]:.3f})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 9. 결론

### 핵심 발견

| 피처 | 타겟과 상관관계 (r) |
|------|-------------------|
| **y_lag1 (전주 판매점수)** | **0.885** |
| y_lag2 (2주전) | 0.776 |
| y_lag3 (3주전) | 0.668 |
| y_lag4 (4주전) | 0.565 |
| 기존 피처 최고 (category_10) | 0.094 |

---

### 다중 모델 성능 비교 요약

#### 회귀 모델 (8개)
| 모델 | 기존 피처만 R² | +y_lag1 R² | y_lag1만 R² |
|------|---------------|------------|-------------|
| **LightGBM** | 0.35 | **0.69** | 0.70 |
| Gradient Boosting | 0.17 | 0.69 | 0.70 |
| Random Forest | 0.42 | 0.67 | 0.70 |
| XGBoost | 0.42 | 0.66 | 0.70 |
| Ridge / Linear | -0.00 | 0.69 | 0.69 |
| Lasso | 0.01 | 0.69 | 0.69 |
| KNN | -0.14 | 0.46 | 0.68 |

#### 분류 모델 (7개)
| 모델 | 기존 피처만 F1 | +y_lag1 F1 | y_lag1만 F1 |
|------|---------------|------------|-------------|
| **Gradient Boosting** | 0.25 | **0.95** | 0.95 |
| Random Forest | 0.34 | 0.95 | 0.95 |
| LightGBM | 0.60 | 0.94 | 0.95 |
| XGBoost | 0.65 | 0.94 | 0.95 |
| Logistic Regression | 0.34 | 0.90 | 0.90 |
| SVM (RBF) | 0.39 | 0.74 | 0.95 |
| KNN | 0.15 | 0.38 | 0.95 |

---

### 피처셋별 평균 성능 (모든 모델)

| 피처셋 | 평균 R² | 평균 F1 | 평균 AUC |
|--------|---------|---------|----------|
| 기존 피처만 | 0.16 | 0.39 | 0.69 |
| **기존 + y_lag1** | **0.66** | **0.83** | **0.91** |
| y_lag1만 | 0.70 | 0.94 | 0.95 |

**y_lag1 추가 시 평균 개선율**: R² +322%, F1 +115%

---

### 최종 결론

1. **y_lag1(전주 판매점수)이 모든 모델에서 가장 강력한 피처**
   - 8개 회귀 모델 평균 R²: 0.16 → 0.66 (+322%)
   - 7개 분류 모델 평균 F1: 0.39 → 0.83 (+115%)

2. **모델 선택보다 피처가 더 중요**
   - y_lag1만 사용해도 대부분의 모델이 R² ≈ 0.70, F1 ≈ 0.95 달성
   - 복잡한 모델(LightGBM) vs 단순 모델(Linear Regression) 성능 차이 미미

3. **"과거 성과가 미래를 결정"**
   - 전주에 베스트셀러였던 책은 이번 주에도 베스트셀러일 확률 높음
   - 뉴스 바이럴, 카테고리 등 외부 변수보다 **과거 실적**이 압도적

4. **실무 적용 권장사항**
   - **기존 책**: y_lag1 + 간단한 모델로 충분
   - **신규 책**: lag 없이 예측하는 별도 모델 필요
   - **앙상블**: lag 모델 + 외부변수 모델 결합 권장